In [11]:
!ls ../data/

Train.zip         titanic-train.csv tmp


In [12]:
!file ../data/titanic-train.csv

../data/titanic-train.csv: ASCII text, with CRLF line terminators


In [13]:
!head ../data/titanic-train.csv

In [14]:
%load_ext autoreload

%autoreload 2
%matplotlib inline

In [15]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from IPython.display import display
from sklearn import metrics

/Users/sameenislam/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/sameenislam/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [30]:
df_raw = pd.read_csv('../data/titanic-train.csv', low_memory=False, index_col='PassengerId')

In [32]:
df_raw.tail().T

PassengerId,887,888,889,890,891
Survived,0,1,0,1,0
Pclass,2,1,3,1,3
Name,"Montvila, Rev. Juozas","Graham, Miss. Margaret Edith","Johnston, Miss. Catherine Helen ""Carrie""","Behr, Mr. Karl Howell","Dooley, Mr. Patrick"
Sex,male,female,female,male,male
Age,27,19,NaN,26,32
SibSp,0,0,1,0,0
Parch,0,0,2,0,0
Ticket,211536,112053,W./C. 6607,111369,370376
Fare,13,30,23.45,30,7.75
Cabin,NaN,B42,NaN,C148,NaN


In [33]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(df_raw.drop('Survived', axis=1), df_raw['Survived'])

ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'

In [ ]:
train_cats(df_raw)